# Memory
How do we remember the previous conversation and feed that into the model so that they can have the conversational flow. LangChain offers multiple sophisticated options of managing these memories.

In [1]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

import warnings
warnings.filterwarnings('ignore')

In [2]:
# account for deprecation of LLM model
import datetime
# Get the current date
current_date = datetime.datetime.now().date()

# Define the date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2024, 6, 12)

# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"

In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

In [4]:
llm = ChatOpenAI(temperature=0.0, model=llm_model)
memory = ConversationBufferMemory()
conversation = ConversationChain(llm=llm, memory = memory, verbose=True)

#### Let's try to build a conversational chain, the green text is the prompt generated by LangChain

In [5]:
conversation.predict(input="Hi, my name is Ebad")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, my name is Ebad
AI:

> Finished chain.


"Hello Ebad, it's nice to meet you. My name is AI. How can I assist you today?"

In [6]:
conversation.predict(input="What is 1+1?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Ebad
AI: Hello Ebad, it's nice to meet you. My name is AI. How can I assist you today?
Human: What is 1+1?
AI:

> Finished chain.


'The answer to 1+1 is 2.'

In [7]:
conversation.predict(input="What is my name?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Ebad
AI: Hello Ebad, it's nice to meet you. My name is AI. How can I assist you today?
Human: What is 1+1?
AI: The answer to 1+1 is 2.
Human: What is my name?
AI:

> Finished chain.


'Your name is Ebad, as you mentioned earlier.'

In [8]:
print(memory.buffer)

Human: Hi, my name is Ebad
AI: Hello Ebad, it's nice to meet you. My name is AI. How can I assist you today?
Human: What is 1+1?
AI: The answer to 1+1 is 2.
Human: What is my name?
AI: Your name is Ebad, as you mentioned earlier.


In [9]:
memory.load_memory_variables({})

{'history': "Human: Hi, my name is Ebad\nAI: Hello Ebad, it's nice to meet you. My name is AI. How can I assist you today?\nHuman: What is 1+1?\nAI: The answer to 1+1 is 2.\nHuman: What is my name?\nAI: Your name is Ebad, as you mentioned earlier."}

## ConversationBufferMemory
The way LangChain is storing the convseration is **ConversationBufferMemory**.

In [10]:
memory = ConversationBufferMemory()

In [11]:
memory.save_context({"input": "Hi"}, 
                    {"output": "What's up"})

Using **`.save_context()`** function we cna keep appending the conversation into the memory explicitly.

In [12]:
print(memory.buffer)

Human: Hi
AI: What's up


In [13]:
memory.load_memory_variables({})

{'history': "Human: Hi\nAI: What's up"}

In [14]:
memory.save_context({"input": "Not much, just hanging"}, 
                    {"output": "Cool"})

In [15]:
memory.load_memory_variables({})

{'history': "Human: Hi\nAI: What's up\nHuman: Not much, just hanging\nAI: Cool"}

When we use a LLM for chat conversation, the LLM itself is actually stateless. The language model itself doesn't remember the conversation we have had so far. And each transaction, each call to the API endpoint is independent. And chatvots appear to have memory only because there is usually rapid code that provides the ful conversation that is been had so far as context to the LLM. And so the memory can sotre explicitly the term or the utterances so far. And this memory storage is used as input or additional context to the LLM so that they can generate an output as if it is just having the next conversational turn, knowing what's been said before. And as th econversation becomes long, the amounts of memory needed becomes really long and does the cost of sending a lot of tokens to the LLM, which usually charges based on the number of tokens it needs to process, will also become more expensive. LangChain provides several convenient kinds of memory to store and accumulate the conversation.

## ConversationBufferWindowMemory
This only keeps an window memory with **`k=1`**, this variable k specifies that we wanted to remember just one conversational exchange. That is one utterance from user and one from the model.

In [16]:
from langchain.memory import ConversationBufferWindowMemory

In [17]:
memory = ConversationBufferWindowMemory(k=1)               

In [18]:
memory.save_context({"input": "Hi"},
                    {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})

In [19]:
memory.load_memory_variables({})

{'history': 'Human: Not much, just hanging\nAI: Cool'}

Now it only remembers the most recent utterance.

In [20]:
llm = ChatOpenAI(temperature=0.0, model=llm_model)
memory = ConversationBufferWindowMemory(k=1)
conversation = ConversationChain(llm=llm, memory = memory, verbose=False)

In [21]:
conversation.predict(input="Hi, my name is Ebad")

"Hello Ebad, it's nice to meet you. My name is AI. How can I assist you today?"

In [22]:
conversation.predict(input="What is 1+1?")

'The answer to 1+1 is 2.'

In [23]:
conversation.predict(input="What is my name?")

"I'm sorry, I don't have access to that information. Could you please tell me your name?"

## ConversationTokenBufferMemory
With this the memory will limit the number of tokens saved, And because a lot of LLM pricing is based on tokens, this map more directly to the cost of the LLM calls. So, if we were to say the **`max_token_limit=50`**.

In [24]:
#!pip install tiktoken

In [25]:
from langchain.memory import ConversationTokenBufferMemory
from langchain.llms import OpenAI
llm = ChatOpenAI(temperature=0.0, model=llm_model)

In [26]:
memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=50)
memory.save_context({"input": "AI is what?!"},
                    {"output": "Amazing!"})
memory.save_context({"input": "Backpropagation is what?"},
                    {"output": "Beautiful!"})
memory.save_context({"input": "Chatbots are what?"}, 
                    {"output": "Charming!"})

In [27]:
memory.load_memory_variables({})

{'history': 'AI: Amazing!\nHuman: Backpropagation is what?\nAI: Beautiful!\nHuman: Chatbots are what?\nAI: Charming!'}

The **`max_token_limit`** chops the content, deletes the older conversation and keeps the recent one.

## ConversationSummaryMemory
Instead of limiting the memory to a fixed number of tokens based on the most recent utterances of a fixed number of conversational exchanges, we can ask LLM to summarize the conversation and let that be the memory.

In [28]:
from langchain.memory import ConversationSummaryBufferMemory

In [29]:
# create a long string
schedule = "There is a meeting at 8am with your product team. \
You will need your powerpoint presentation prepared. \
9am-12pm have time to work on your LangChain \
project which will go quickly because Langchain is such a powerful tool. \
At Noon, lunch at the italian resturant with a customer who is driving \
from over an hour away to meet you to understand the latest in AI. \
Be sure to bring your laptop to show the latest LLM demo."

memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100)
memory.save_context({"input": "Hello"}, {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})
memory.save_context({"input": "What is on the schedule today?"}, 
                    {"output": f"{schedule}"})

In [30]:
memory.load_memory_variables({})

{'history': "System: The human and AI engage in small talk before discussing the day's schedule. The AI informs the human of a morning meeting with the product team, time to work on the LangChain project, and a lunch meeting with a customer interested in the latest AI developments."}

Due to the low value of **`max_token_limit`** the model summarize the entire conversation to fit the context within the limit.

In [31]:
conversation = ConversationChain(llm=llm, memory = memory, verbose=True)

In [32]:
conversation.predict(input="What would be a good demo to show?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: The human and AI engage in small talk before discussing the day's schedule. The AI informs the human of a morning meeting with the product team, time to work on the LangChain project, and a lunch meeting with a customer interested in the latest AI developments.
Human: What would be a good demo to show?
AI:

> Finished chain.


"Based on the customer's interest in AI developments, I would suggest showcasing our latest natural language processing capabilities. We could demonstrate how our AI can understand and respond to complex language queries, and even provide personalized recommendations based on the user's preferences. Additionally, we could highlight our machine learning algorithms and how they continuously improve the accuracy and efficiency of our AI. Would you like me to prepare a demo for the meeting?"

In [33]:
memory.load_memory_variables({})

{'history': "System: The human and AI engage in small talk before discussing the day's schedule. The AI informs the human of a morning meeting with the product team, time to work on the LangChain project, and a lunch meeting with a customer interested in the latest AI developments. The human asks what would be a good demo to show.\nAI: Based on the customer's interest in AI developments, I would suggest showcasing our latest natural language processing capabilities. We could demonstrate how our AI can understand and respond to complex language queries, and even provide personalized recommendations based on the user's preferences. Additionally, we could highlight our machine learning algorithms and how they continuously improve the accuracy and efficiency of our AI. Would you like me to prepare a demo for the meeting?"}

#### The overall summary is stored as the system message.

# Memory Types
- **ConversationalBufferMemory :-** This memory allows for storing of messages and then extracts the messages in a variable.

- **ConversationalBufferWindowMemory :-** This memory keeps a list of the interactions of the conversation over time. It only uses the last K interactions.

- **ConversationalTokenBufferMemory :-** This memory keeps a buffer of recent interactions in memory, and uses token length rather than number of interactions to determine when to flush interactions.

- **ConversationalSummaryMemory :-** This memory creates a summary of the conversation over time.

# Additional Memory Types
- **Vector Data Memory :-** Stores text (from conversation or elsewhere) in a vector database and retrieves the most relevant blocks of text.

- **Entity Memories :-** Using an LLM, it remembers details about specific entities.

We can also use multiple memories at one time. For example Conversation memory + Entitiy memory to recall individuals. We can also store the conversation in a conversational database such as **key-value** store or **SQL**.